In [11]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot


import h5py
import pandas as pd

# Wczytywanie pliku

In [12]:

f = h5py.File('C:/Users/huboz/Desktop/Pjatk/HARDataset/hma_basics_with_analog_preprocesing.h5', 'r')


In [29]:
f

<HDF5 file "hma_basics_with_analog_preprocesing.h5" (mode r)>

In [13]:
# ładuje listę plików do trójwymiarowej tablicy X=samples, timesteps, features (Dane wejściowe), y= samples,label (dane wyjściowe)

x = f['x']
y = f['y']
print('(samples, timesteps, features) (samples,label) -> ', x.shape, y.shape)

(samples, timesteps, features) (samples,label) ->  (1991, 113, 184) (1991, 1)


In [14]:
x.shape, y.shape

((1991, 113, 184), (1991, 1))

## (1394,113,184)(1394,1) - Train
## (497,113,184)(497,1)-Test

In [15]:
def trenujX():
    x1=[]
    for i in range(0, 1394):
        x1.append(x[i][112][183])

    return x1

In [16]:
def trenujY():
    y1=[]
    for i in range(0, 1394):
        y1.append(y[i][0])
    
    return y1

In [17]:
def testX():
    x2=[]
    for i in range(1395,1990):
        x2.append(x[i][112][183])

    return x2

In [18]:
def testY():
    y2=[]
    for i in range(1395,1990):
        y2.append(y[i][0])
    
    return y2

### załaduje listę plików i zwróci jako trójwymiarową tablicę numpy

In [19]:

def load_group(filenames):
    loaded = list()
    for name in filenames:
        data = f
        loaded.append(data)
    # grupę stosów, tak aby funkcje były trzecim wymiarem
    loaded = dstack(loaded)
    return loaded



# Ładujemy dane

In [20]:
import numpy as np

X = x[0:1991]
Y = y[0:1991]

def Shuffle_Arr(arr):
    np.random.shuffle(arr)
    
    return arr

In [21]:
def load_dataset():
    
    Shuffle_Arr(X)
    Shuffle_Arr(Y)
    
    TrainX = X[0:1394]
    TestX = X[1395:1991]
    TrainY = Y[0:1394]
    TestY = Y[1395:1991]

    TrainY = to_categorical(TrainY)
    TestY = to_categorical(TestY)
    #print(trenujX.shape, trenujY.shape, testX.shape, testY.shape)
    return TrainX, TrainY, TestX, TestY

In [22]:
TrainX, TrainY, TestX, TestY = load_dataset()

# struktura sieci

In [23]:

# def evaluate_model(TrainX, TrainY, TestX, TestY):
# 	# define model
# 	verbose, epochs, batch_size = 0, 25, 64
# 	n_timesteps, n_features, n_outputs = TrainX.shape[1], TrainX.shape[2], TrainY.shape[1]
# 	# przekształcić w trening (próbki, kroki czasowe, rzędy, kolumny, kanały)
# 	n_steps, n_length = 4, 32
# 	TrainX = TrainX.reshape((TrainX.shape[0], n_steps, 1, n_length, n_features))
# 	TestX = TestX.reshape((TestX.shape[0], n_steps, 1, n_length, n_features))
# 	# define model
# 	model = Sequential()
# 	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
# 	model.add(Dropout(0.5))
# 	model.add(Flatten())
# 	model.add(Dense(100, activation='relu'))
# 	model.add(Dense(n_outputs, activation='softmax'))
# 	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	# fit network
# 	model.fit(TrainX, TrainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
# 	# evaluate model
# 	_, accuracy = model.evaluate(x2, y2, batch_size=batch_size, verbose=0)
# 	return accuracy


In [24]:
def evaluate_model(TrainX, TrainY, TestX, TestY):
    verbose, epochs, batch_size = 0, 10, 64 # 15, 64 domyslnie, dla 10, 64 działa najlepiej
    n_timesteps, n_features, n_outputs = TrainX.shape[1], TrainX.shape[2], TrainY.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.1)) # 0.5
    model.add(Flatten()) #dodalam
    model.add(Dropout(0.5)) #dodalam
    model.add(Flatten()) #dodalam
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(TrainX, TrainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(TestX, TestY, batch_size=batch_size, verbose=0)
    return accuracy

In [30]:
evaluate_model(TrainX, TrainY, TestX, TestY)

0.33892616629600525

In [26]:
# podsumowuje wyniki uruchomienia

def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [27]:
def run_experiment(repeats=10):
    # load data
    TrainX, TrainY, testX, TestY = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(TrainX, TrainY, TestX, TestY)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)


In [28]:
run_experiment(repeats=10)

>#1: 36.074
>#2: 34.564
>#3: 33.893
>#4: 32.047
>#5: 31.208
>#6: 31.711
>#7: 32.550
>#8: 31.208
>#9: 30.872
>#10: 32.047
[36.073824763298035, 34.563758969306946, 33.892616629600525, 32.046979665756226, 31.20805323123932, 31.711408495903015, 32.55033493041992, 31.20805323123932, 30.87248206138611, 32.046979665756226]
Accuracy: 32.617% (+/-1.608)
